# Import Modules

In [1]:
import importlib
import os
import sys

import joblib
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'polars'

In [ ]:
os.chdir("../")
sys.path.insert(0, os.getcwd())

In [ ]:
from morai.experience import charters, experience
from morai.forecast import metrics, preprocessors
from morai.models import base, gam
from morai.utils import custom_logger, helpers

In [ ]:
logger = custom_logger.setup_logging(__name__)

In [ ]:
# update log level if wanting more logging
custom_logger.set_log_level("INFO")

In [ ]:
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
# default is "plotly_mimetype+notebook", however that takes up space.
# "plotly_mimetype+notebook_connected" seems to save space
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

# Data

In [ ]:
pl_parquet_path = r"files/dataset/mortality_grouped.parquet"

In [ ]:
# reading in the dataset
# `enable_string_cache` helps with categorical type values
pl.enable_string_cache()
lzdf = pl.scan_parquet(
    pl_parquet_path,
)

In [ ]:
initial_row_count = lzdf.select(pl.len()).collect().item()
print(
    f"row count: {initial_row_count:,} \n"
    f"exposures: {lzdf.select([pl.col('amount_exposed').sum()]).collect()[0,0]:,}"
)

row count: 1,793,371 
exposures: 9,824,025,879,572.559


In [ ]:
grouped_df = lzdf.collect()

In [ ]:
grouped_df = grouped_df.to_pandas()

# Preparing Data

## Filter

In [ ]:
model_data = grouped_df[
    (grouped_df["attained_age"] >= 50)
    # & (grouped_df["attained_age"] <= 95)
    & (grouped_df["issue_age"] >= 30)
    & (grouped_df["issue_age"] <= 80)
].copy()
model_data = model_data.reset_index(drop=True)

In [ ]:
del grouped_df

## Calculated Fields

In [ ]:
model_data["capped_duration"] = model_data["duration"].clip(upper=26)
model_data["qx_log_raw"] = np.log(model_data["qx_raw"] + 1)
binned_face_dict = {
    "01: 0 - 9,999": "01: 0 - 24,999",
    "02: 10,000 - 24,999": "01: 0 - 24,999",
    "03: 25,000 - 49,999": "02: 25,000 - 99,999",
    "04: 50,000 - 99,999": "02: 25,000 - 99,999",
    "05: 100,000 - 249,999": "03: 100,000 - 249,999",
    "06: 250,000 - 499,999": "04: 250,000 - 4,999,999",
    "07: 500,000 - 999,999": "04: 250,000 - 4,999,999",
    "08: 1,000,000 - 2,499,999": "04: 250,000 - 4,999,999",
    "09: 2,500,000 - 4,999,999": "04: 250,000 - 4,999,999",
    "10: 5,000,000 - 9,999,999": "05: 5,000,000+",
    "11: 10,000,000+": "05: 5,000,000+",
}
model_data["binned_face"] = model_data["face_amount_band"].map(binned_face_dict)
model_data["binned_face"] = model_data["binned_face"].astype("category")

## Feature Dictionary

In [ ]:
feature_dict = {
    "target": ["qx_raw"],
    "weight": ["amount_exposed"],
    "passthrough": ["attained_age", "duration", "observation_year"],
    "ordinal": [
        "sex",
        "smoker_status",
    ],
    "ohe": [
        "binned_face",
        "insurance_plan",
        "class_enh",
    ],
    "nominal": [],
}

feature_dict_vbt = {
    "target": ["qx_raw"],
    "weight": ["amount_exposed"],
    "passthrough": ["attained_age", "capped_duration"],
    "ordinal": [
        "sex",
        "smoker_status",
    ],
    "ohe": [],
    "nominal": [],
}

## Model Results Dictionary

In [ ]:
metric_cols = ["ae", "smape", "r2_score", "root_mean_squared_error", "aic", "shape"]
model_results = metrics.ModelResults(metrics=metric_cols)

### VBT15

In [ ]:
model_name = "vbt15"

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=model_data["death_claim_amount"],
    y_pred_train=model_data[f"exp_amt_{model_name}"],
    weights_train=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=None,
    model_params=None,
    scorecard=scorecard,
    importance=None,
)

 2025-05-12 23:08:03 | morai.forecast.metrics | INFO     | Adding model 'vbt15' 


# Forecasting Models

In [ ]:
# # build
model_build = True
model_load = False
model_save = True

In [ ]:
# load
model_build = False
model_load = True
model_save = False

## GLM

**Overview:**
- GLM is a flexible generalization of ordinary linear regression models. It supports non-normal distributions of the dependent variable.

**Feature Preprocessing:**
  - GLM - using one-hot enconding (ohe) as the model needs the categories to be diferentiated and not ordinal or nominal. This only applies to non-binary features.
  - Scaling should be considered to ensure the features aren't overly weighted by larger values.

**Model Characteristics**
  - GLM is a simpler type of model and is easily understandable. May struggle with complex, non-linear interactions without suitable transformations or feature engineering.

In [ ]:
model_name = "glm"

In [ ]:
preprocess_dict = preprocessors.preprocess_data(
    model_data,
    feature_dict=feature_dict,
    standardize=False,
    add_constant=True,
)

 2025-05-13 00:04:08 | morai.forecast.preprocessors | INFO     | model target: ['qx_raw'] 
 2025-05-13 00:04:08 | morai.forecast.preprocessors | INFO     | model weights: ['amount_exposed'] 
 2025-05-13 00:04:08 | morai.forecast.preprocessors | INFO     | adding a constant column to the data 
 2025-05-13 00:04:08 | morai.forecast.preprocessors | INFO     | passthrough - (generally numeric): ['attained_age', 'observation_year', 'duration', 'constant'] 
 2025-05-13 00:04:08 | morai.forecast.preprocessors | INFO     | ordinal - ordinal encoded: ['sex', 'smoker_status'] 
 2025-05-13 00:04:09 | morai.forecast.preprocessors | INFO     | nominal - one hot encoded (dropping first col): ['binned_face', 'class_enh', 'insurance_plan'] 


In [ ]:
X = preprocess_dict["X"]
y = preprocess_dict["y"]
weights = preprocess_dict["weights"]
mapping = preprocess_dict["mapping"]
md_encoded = preprocess_dict["md_encoded"]
model_features = preprocess_dict["model_features"]

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, random_state=0, test_size=0.2
)

In [ ]:
GLM = base.GLM()
if model_build:
    GLM.fit(X=X_train, y=y_train, weights=weights_train, r_style=False)
if model_load:
    GLM.model = joblib.load(f"files/models/{model_name}.joblib")
    logger.info(f"loaded model '{model_name}'. type: {type(GLM.model)}")
    GLM.is_fitted_ = True
if model_save:
    joblib.dump(GLM.model, f"files/models/{model_name}.joblib")
    logger.info(f"saved model '{model_name}'. type: {type(GLM.model)}")

model_params = {"weights": True, "r_style": False}
model_params.update({"family": GLM.model.family})

 2025-05-13 00:04:11 | morai.models.base | INFO     | fiting the model 
 2025-05-13 00:04:11 | morai.models.base | INFO     | setup GLM model with statsmodels and <statsmodels.genmod.families.family.Binomial object at 0x000001C30F542420> family... 
 2025-05-13 00:04:30 | __main__ | INFO     | saved model 'glm'. type: <class 'statsmodels.genmod.generalized_linear_model.GLMResultsWrapper'> 


In [ ]:
print(GLM.model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 qx_raw   No. Observations:               877381
Model:                            GLM   Df Residuals:         3418917283149.29
Model Family:                Binomial   Df Model:                           24
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -7.5391e+10
Date:                Tue, 13 May 2025   Deviance:                   7.4554e+10
Time:                        00:04:31   Pearson chi2:                 5.27e+11
No. Iterations:                    10   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
at

In [ ]:
predictions = GLM.predict(X)
print(f"NA values: {np.isnan(predictions).sum()}")

NA values: 0


In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
odds = GLM.get_odds(display=False)
importance = base.ModelWrapper(GLM.model).get_importance()

 2025-05-13 00:04:32 | morai.models.base | INFO     | generating odds ratio from model 


In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=y_train,
    y_pred_train=GLM.model.predict(X_train),
    weights_train=weights_train,
    y_true_test=y_test,
    y_pred_test=GLM.model.predict(X_test),
    weights_test=weights_test,
    model=GLM.model,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=preprocess_dict,
    model_params=model_params,
    scorecard=scorecard,
    importance=importance,
)

 2025-05-13 00:04:32 | morai.forecast.metrics | INFO     | Adding model 'glm' 


In [ ]:
charters.compare_rates(
    df=model_data[model_data["insurance_plan"].isin(["UL", "ULSG"])],
    x_axis="duration",
    rates=["ae_glm", "ae_vbt15"],
    weights=["exp_amt_glm", "exp_amt_vbt15"],
    secondary="death_count",
    x_bins=6,
    display=True,
)

 2025-05-13 00:04:33 | morai.experience.charters | INFO     | Binning feature: [duration] with 6 bins 


In [ ]:
charters.compare_rates(
    model_data[model_data["insurance_plan"].isin(["UL"])],
    x_axis="attained_age",
    rates=["qx_raw", "qx_vbt15", "qx_glm"],
    weights=["amount_exposed"],
    secondary="death_count",
    y_log=False,
)

 2025-05-13 00:04:33 | morai.experience.charters | INFO     | The weights list is 1 long and should be 3 long. Using the first weight for all weights. 


In [ ]:
charters.target(
    df=model_data.loc[X_train.index],
    target="ratio",
    cols=3,
    features=model_features,
    numerator=["death_claim_amount"],
    denominator=["exp_amt_glm"],
).show()

 2025-05-13 00:04:34 | morai.experience.charters | INFO     | Creating '10' target plots. 


## GAM

**Overview:**
- Generalized Additive Models (GAMs) are a class of regression models that combine the interpretability of linear models with the flexibility of non-linear models by fitting separate smooth functions to each feature.
  
**Feature Preprocessing:**
  - GAMs can handle both numeric and categorical features, but numeric features may require scaling or transformation for optimal results.

**Model Characteristics**
  - GAMs handle non-linear relationships effectively through smooth functions while maintaining interpretability.
  - Unlike tree-based models, GAMs do not split data but instead apply smooth functions to individual predictors.

In [ ]:
model_name = "gam"

In [ ]:
preprocess_dict = preprocessors.preprocess_data(
    model_data,
    feature_dict=feature_dict,
    standardize=False,
    add_constant=True,
)

 2025-05-12 23:18:08 | morai.forecast.preprocessors | INFO     | model target: ['qx_raw'] 
 2025-05-12 23:18:08 | morai.forecast.preprocessors | INFO     | model weights: ['amount_exposed'] 
 2025-05-12 23:18:08 | morai.forecast.preprocessors | INFO     | adding a constant column to the data 
 2025-05-12 23:18:08 | morai.forecast.preprocessors | INFO     | passthrough - (generally numeric): ['attained_age', 'observation_year', 'duration', 'constant'] 
 2025-05-12 23:18:08 | morai.forecast.preprocessors | INFO     | ordinal - ordinal encoded: ['sex', 'smoker_status'] 
 2025-05-12 23:18:09 | morai.forecast.preprocessors | INFO     | nominal - one hot encoded (dropping first col): ['binned_face', 'class_enh', 'insurance_plan'] 


In [ ]:
X = preprocess_dict["X"]
y = preprocess_dict["y"]
weights = preprocess_dict["weights"]
mapping = preprocess_dict["mapping"]
md_encoded = preprocess_dict["md_encoded"]
model_features = preprocess_dict["model_features"]

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, random_state=0, test_size=0.2
)

In [ ]:
GAM = gam.GAMR()
spline_dict = {
    "attained_age": {"df":12, "degree": 3, "bs":"ps"},
}
if model_build:
    GAM.setup_model(
        X=X_train, y=y_train, weights=weights_train, spline_dict=spline_dict
    )
if model_load:
    GAM.model = joblib.load(f"files/models/{model_name}.joblib")
    logger.info(f"loaded model '{model_name}'. type: {type(GAM.model)}")
    GAM.is_fitted_ = True
if model_save:
    joblib.dump(GAM.model, f"files/models/{model_name}.joblib")
    logger.info(f"saved model '{model_name}'. type: {type(GAM.model)}")

model_params = {"weights": True}
model_params.update({"family": GAM.family})
model_params.update({"splne_dict": spline_dict})

 2025-05-12 23:18:11 | morai.models.gam | INFO     | setup GAM model with mgcv and `quasibinomial` distribution with `logit` link 
 2025-05-12 23:18:17 | morai.models.gam | INFO     | formula: 
 2025-05-12 23:18:17 | morai.models.gam | INFO     | > qx_raw ~ s(`attained_age`, k=12, m=3, bs='ps')+`binned_face_02__25_000___99_999`+`binned_face_03__100_000___249_999`+`binned_face_04__250_000___4_999_999`+`binned_face_05__5_000_000plus`+`class_enh_2_2`+`class_enh_3_1`+`class_enh_3_2`+`class_enh_3_3`+`class_enh_4_1`+`class_enh_4_2`+`class_enh_4_3`+`class_enh_4_4`+`class_enh_NA_NA`+`class_enh_U_U`+`constant`+`duration`+`insurance_plan_Term`+`insurance_plan_UL`+`insurance_plan_ULSG`+`insurance_plan_VL`+`insurance_plan_VLSG`+`observation_year`+`sex`+`smoker_status` 
 2025-05-12 23:21:36 | morai.models.gam | INFO     | fitting the model: 
 2025-05-12 23:21:36 | morai.models.gam | INFO     | > model <- bam(formula, data=data, weights=weights, family=family, drop.intercept=TRUE)` 
 2025-05-12 23:2

In [ ]:
print(GAM.summary(expand=True))

Generalized Additive Model (mgcv) Summary
Family                : quasibinomial
Link                  : logit
Number of Observations: 877381
Adjusted R-squared    : 0.104
Deviance Explained    : 32.5%
Scale Estimate        : 5.751e+05
fREML                 : 7.063e+06

Formula:
qx_raw ~ s(`attained_age`, k=12, m=3, bs='ps')+`binned_face_02__25_000___99_999`+`binned_face_03__100_000___249_999`+`binned_face_04__250_000___4_999_999`+`binned_face_05__5_000_000plus`+`class_enh_2_2`+`class_enh_3_1`+`class_enh_3_2`+`class_enh_3_3`+`class_enh_4_1`+`class_enh_4_2`+`class_enh_4_3`+`class_enh_4_4`+`class_enh_NA_NA`+`class_enh_U_U`+`constant`+`duration`+`insurance_plan_Term`+`insurance_plan_UL`+`insurance_plan_ULSG`+`insurance_plan_VL`+`insurance_plan_VLSG`+`observation_year`+`sex`+`smoker_status`

Parametric Coefficients:
binned_face_02__25_000___99_999       -0.11
binned_face_03__100_000___249_999     -0.21
binned_face_04__250_000___4_999_999   -0.30
binned_face_05__5_000_000plus         -0.50
c

In [ ]:
predictions = GAM.predict(X)

 2025-05-12 23:27:12 | morai.models.gam | INFO     | predicted rates 


In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=y_train,
    y_pred_train=GAM.predict(X_train),
    weights_train=weights_train,
    y_true_test=y_test,
    y_pred_test=GAM.predict(X_test),
    weights_test=weights_test,
    model=GAM.model,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=preprocess_dict,
    model_params=model_params,
    scorecard=scorecard,
    importance=None,
)

 2025-05-12 23:30:24 | morai.models.gam | INFO     | predicted rates 
 2025-05-12 23:31:13 | morai.models.gam | INFO     | predicted rates 
 2025-05-12 23:31:13 | morai.forecast.metrics | ERROR    | Model `
Family: quasibinomial 
Link function: logit 

Formula:
qx_raw ~ s(attained_age, k = 12, m = 3, bs = "ps") + binned_face_02__25_000___99_999 + 
    binned_face_03__100_000___249_999 + binned_face_04__250_000___4_999_999 + 
    binned_face_05__5_000_000plus + class_enh_2_2 + class_enh_3_1 + 
    class_enh_3_2 + class_enh_3_3 + class_enh_4_1 + class_enh_4_2 + 
    class_enh_4_3 + class_enh_4_4 + class_enh_NA_NA + class_enh_U_U + 
    constant + duration + insurance_plan_Term + insurance_plan_UL + 
    insurance_plan_ULSG + insurance_plan_VL + insurance_plan_VLSG + 
    observation_year + sex + smoker_status

Estimated degrees of freedom:
6.67  total = 30.67 

fREML score: 7063080     
` does not have AIC attribute, returning None 
 2025-05-12 23:31:13 | morai.forecast.metrics | ERROR  

In [ ]:
charters.compare_rates(
    model_data,
    x_axis="attained_age",
    rates=["qx_raw", "qx_vbt15", "qx_gam"],
    weights=["amount_exposed"],
    secondary="death_count",
)

 2025-05-12 23:31:14 | morai.experience.charters | INFO     | The weights list is 1 long and should be 3 long. Using the first weight for all weights. 


## Linear Regression

**Feature Preprocessing:**
- Linear regression needs all features to be numeric.

In [ ]:
model_name = "lr"

In [ ]:
preprocess_dict = preprocessors.preprocess_data(
    model_data,
    feature_dict=feature_dict,
    standardize=False,
)

 2025-05-12 23:31:15 | morai.forecast.preprocessors | INFO     | model target: ['qx_raw'] 
 2025-05-12 23:31:15 | morai.forecast.preprocessors | INFO     | model weights: ['amount_exposed'] 
 2025-05-12 23:31:15 | morai.forecast.preprocessors | INFO     | passthrough - (generally numeric): ['attained_age', 'observation_year', 'duration'] 
 2025-05-12 23:31:15 | morai.forecast.preprocessors | INFO     | ordinal - ordinal encoded: ['sex', 'smoker_status'] 
 2025-05-12 23:31:16 | morai.forecast.preprocessors | INFO     | nominal - one hot encoded (dropping first col): ['binned_face', 'class_enh', 'insurance_plan'] 


In [ ]:
X = preprocess_dict["X"]
y = preprocess_dict["y"]
weights = preprocess_dict["weights"]
mapping = preprocess_dict["mapping"]
md_encoded = preprocess_dict["md_encoded"]
model_features = preprocess_dict["model_features"]

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, random_state=0, test_size=0.2
)

In [ ]:
from sklearn import linear_model

if model_build:
    model = linear_model.LinearRegression()
    model.fit(X_train, y_train, sample_weight=weights_train)
if model_load:
    model = joblib.load(f"files/models/{model_name}.joblib")
    logger.info(f"loaded model '{model_name}'. type: {type(model)}")
if model_save:
    joblib.dump(model, f"files/models/{model_name}.joblib")
    logger.info(f"saved model '{model_name}'. type: {type(model)}")

model_params = {"weights": True}
model_params.update(model.get_params())

 2025-05-12 23:31:19 | __main__ | INFO     | saved model 'lr'. type: <class 'sklearn.linear_model._base.LinearRegression'> 


In [ ]:
predictions = model.predict(X)

In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
importance = base.ModelWrapper(model).get_importance()

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=y_train,
    y_pred_train=model.predict(X_train),
    weights_train=weights_train,
    y_true_test=y_test,
    y_pred_test=model.predict(X_test),
    weights_test=weights_test,
    model=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=preprocess_dict,
    model_params=model_params,
    scorecard=scorecard,
    importance=importance,
)

 2025-05-12 23:31:20 | morai.forecast.metrics | ERROR    | Model name lr already exists, not adding model. Please use a different name. 


In [ ]:
charters.compare_rates(
    model_data,
    x_axis="attained_age",
    rates=["qx_raw", "qx_vbt15", "qx_lr"],
    weights=["amount_exposed"],
    secondary="death_count",
)

 2025-05-12 23:31:20 | morai.experience.charters | INFO     | The weights list is 1 long and should be 3 long. Using the first weight for all weights. 


In [ ]:
charters.chart(
    df=importance, x_axis="feature", y_axis="importance", type="bar", y_sort=True
)

In [ ]:
charters.target(
    df=model_data,
    target="ratio",
    cols=3,
    features=model_features,
    numerator=["death_claim_amount"],
    denominator=["exp_amt_lr"],
)

 2025-05-12 23:31:21 | morai.experience.charters | INFO     | Creating '9' target plots. 


## Decision Tree

**Feature Preprocessing:**
  - Tree is using ordinal as there is no categorical for this implementation of decision tree
  - Scaling is not necessary in decision trees as the splits are done at certain points and not scaled

In [ ]:
model_name = "tree"

In [ ]:
preprocess_dict = preprocessors.preprocess_data(
    model_data,
    feature_dict=feature_dict,
    standardize=False,
    preset="tree",
)

 2025-05-12 23:31:22 | morai.forecast.preprocessors | INFO     | using 'tree' preset which doesn't need to use 'nominal' or 'ohe' and instead uses 'ordinal' 
 2025-05-12 23:31:22 | morai.forecast.preprocessors | INFO     | model target: ['qx_raw'] 
 2025-05-12 23:31:22 | morai.forecast.preprocessors | INFO     | model weights: ['amount_exposed'] 
 2025-05-12 23:31:22 | morai.forecast.preprocessors | INFO     | passthrough - (generally numeric): ['attained_age', 'observation_year', 'duration'] 
 2025-05-12 23:31:22 | morai.forecast.preprocessors | INFO     | ordinal - ordinal encoded: ['sex', 'smoker_status', 'binned_face', 'class_enh', 'insurance_plan'] 


In [ ]:
X = preprocess_dict["X"]
y = preprocess_dict["y"]
weights = preprocess_dict["weights"]
mapping = preprocess_dict["mapping"]
md_encoded = preprocess_dict["md_encoded"]
model_features = preprocess_dict["model_features"]

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, random_state=0, test_size=0.2
)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

if model_build:
    model = DecisionTreeRegressor(max_depth=6)
    model.fit(X_train, y_train, sample_weight=weights_train)
if model_load:
    model = joblib.load(f"files/models/{model_name}.joblib")
    logger.info(f"loaded model '{model_name}'. type: {type(model)}")
if model_save:
    joblib.dump(model, f"files/models/{model_name}.joblib")
    logger.info(f"saved model '{model_name}'. type: {type(model)}")
model_params = {"weights": True}
model_params.update(model.get_params())

 2025-05-12 23:31:28 | __main__ | INFO     | saved model 'tree'. type: <class 'sklearn.tree._classes.DecisionTreeRegressor'> 


In [ ]:
predictions = model.predict(X)

In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
importance = base.ModelWrapper(model).get_importance()

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=y_train,
    y_pred_train=model.predict(X_train),
    weights_train=weights_train,
    y_true_test=y_test,
    y_pred_test=model.predict(X_test),
    weights_test=weights_test,
    model=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=preprocess_dict,
    model_params=model_params,
    scorecard=scorecard,
    importance=importance,
)

 2025-05-12 23:31:29 | morai.forecast.metrics | ERROR    | Model name tree already exists, not adding model. Please use a different name. 


In [ ]:
charters.chart(
    df=importance, x_axis="feature", y_axis="importance", type="bar", y_sort=True
)

In [ ]:
charters.compare_rates(
    model_data,
    x_axis="attained_age",
    rates=["qx_raw", "qx_vbt15", "qx_tree"],
    weights=["amount_exposed"],
    secondary="death_count",
)

 2025-05-12 23:31:30 | morai.experience.charters | INFO     | The weights list is 1 long and should be 3 long. Using the first weight for all weights. 


In [ ]:
charters.target(
    df=model_data.loc[X_train.index],
    target="ratio",
    cols=3,
    features=model_features,
    numerator=["death_claim_amount"],
    denominator=["exp_amt_tree"],
).show()

 2025-05-12 23:31:31 | morai.experience.charters | INFO     | Creating '9' target plots. 


## Random Forest

**Feature Preprocessing:**
  - Tree is using ordinal as there is no categorical for this implementation of decision tree
  - Scaling is not necessary in decision trees as the splits are done at certain points and not scaled

**Model Characteristics**
  - Random Forest is good at finding non-linear relationships and is a `bagging` type model which limits overfitting.

In [ ]:
model_name = "rf"

In [ ]:
preprocess_dict = preprocessors.preprocess_data(
    model_data,
    feature_dict=feature_dict,
    standardize=False,
    preset="tree",
)

 2025-05-12 23:31:32 | morai.forecast.preprocessors | INFO     | using 'tree' preset which doesn't need to use 'nominal' or 'ohe' and instead uses 'ordinal' 
 2025-05-12 23:31:32 | morai.forecast.preprocessors | INFO     | model target: ['qx_raw'] 
 2025-05-12 23:31:32 | morai.forecast.preprocessors | INFO     | model weights: ['amount_exposed'] 
 2025-05-12 23:31:32 | morai.forecast.preprocessors | INFO     | passthrough - (generally numeric): ['attained_age', 'observation_year', 'duration'] 
 2025-05-12 23:31:32 | morai.forecast.preprocessors | INFO     | ordinal - ordinal encoded: ['sex', 'smoker_status', 'binned_face', 'class_enh', 'insurance_plan'] 


In [ ]:
X = preprocess_dict["X"]
y = preprocess_dict["y"]
weights = preprocess_dict["weights"]
mapping = preprocess_dict["mapping"]
md_encoded = preprocess_dict["md_encoded"]
model_features = preprocess_dict["model_features"]

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, random_state=0, test_size=0.2
)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

if model_build:
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=5,
        oob_score=True,
        random_state=42,
    )

    model.fit(X_train, y_train, sample_weight=weights_train)
if model_load:
    model = joblib.load(f"files/models/{model_name}.joblib")
    logger.info(f"loaded model '{model_name}'. type: {type(model)}")
if model_save:
    joblib.dump(model, f"files/models/{model_name}.joblib")
    logger.info(f"saved model '{model_name}'. type: {type(model)}")

model_params = {"weights": True}
model_params.update(model.get_params())

 2025-05-12 23:38:27 | __main__ | INFO     | saved model 'rf'. type: <class 'sklearn.ensemble._forest.RandomForestRegressor'> 


In [ ]:
predictions = model.predict(X)

In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
importance = base.ModelWrapper(model).get_importance()

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=y_train,
    y_pred_train=model.predict(X_train),
    weights_train=weights_train,
    y_true_test=y_test,
    y_pred_test=model.predict(X_test),
    weights_test=weights_test,
    model=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=preprocess_dict,
    model_params=model_params,
    scorecard=scorecard,
    importance=importance,
)

 2025-05-12 23:39:37 | morai.forecast.metrics | ERROR    | Model name rf already exists, not adding model. Please use a different name. 


In [ ]:
charters.chart(
    df=importance, x_axis="feature", y_axis="importance", type="bar", y_sort=True
)

In [ ]:
charters.compare_rates(
    df=model_data[model_data["insurance_plan"].isin(["UL", "ULSG"])],
    x_axis="duration",
    rates=["ae_rf", "ae_vbt15"],
    weights=["exp_amt_rf", "exp_amt_vbt15"],
    secondary="death_count",
    x_bins=6,
    display=True,
)

 2025-05-12 23:39:38 | morai.experience.charters | INFO     | Binning feature: [duration] with 6 bins 


In [ ]:
charters.target(
    df=model_data,
    target="ratio",
    cols=3,
    features=model_features + ["duration", "issue_year"],
    numerator=["death_claim_amount"],
    denominator=["exp_amt_rf"],
).show()

 2025-05-12 23:39:38 | morai.experience.charters | INFO     | Creating '11' target plots. 


## GBM

**Overview:**
- An ensemble technique that builds models sequentially, each new model aiming to reduce the errors of previous ones. This is base boosting.
  
**Feature Preprocessing:**
  - GBM needs all features to be numeric.

**Model Characteristics**
  - GBM handles non-linear relationships but more difficult to understand
  - It is similar to a random forest, however this builds trees sequentially.

In [ ]:
model_name = "gbm"

In [ ]:
preprocess_dict = preprocessors.preprocess_data(
    model_data,
    feature_dict=feature_dict,
    standardize=False,
)

 2025-05-12 23:39:39 | morai.forecast.preprocessors | INFO     | model target: ['qx_raw'] 
 2025-05-12 23:39:39 | morai.forecast.preprocessors | INFO     | model weights: ['amount_exposed'] 
 2025-05-12 23:39:39 | morai.forecast.preprocessors | INFO     | passthrough - (generally numeric): ['attained_age', 'observation_year', 'duration'] 
 2025-05-12 23:39:39 | morai.forecast.preprocessors | INFO     | ordinal - ordinal encoded: ['sex', 'smoker_status'] 
 2025-05-12 23:39:40 | morai.forecast.preprocessors | INFO     | nominal - one hot encoded (dropping first col): ['binned_face', 'class_enh', 'insurance_plan'] 


In [ ]:
X = preprocess_dict["X"]
y = preprocess_dict["y"]
weights = preprocess_dict["weights"]
mapping = preprocess_dict["mapping"]
md_encoded = preprocess_dict["md_encoded"]
model_features = preprocess_dict["model_features"]

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, random_state=0, test_size=0.2
)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

if model_build:
    model = GradientBoostingRegressor(
        n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42
    )
    model.fit(X_train, y_train, sample_weight=weights_train)
if model_load:
    model = joblib.load(f"files/models/{model_name}.joblib")
    logger.info(f"loaded model '{model_name}'. type: {type(model)}")
if model_save:
    joblib.dump(model, f"files/models/{model_name}.joblib")
    logger.info(f"saved model '{model_name}'. type: {type(model)}")

model_params = {"weights": True}
model_params.update(model.get_params())

 2025-05-12 23:43:06 | __main__ | INFO     | saved model 'gbm'. type: <class 'sklearn.ensemble._gb.GradientBoostingRegressor'> 


In [ ]:
predictions = model.predict(X)

In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
importance = base.ModelWrapper(model).get_importance()

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=y_train,
    y_pred_train=model.predict(X_train),
    weights_train=weights_train,
    y_true_test=y_test,
    y_pred_test=model.predict(X_test),
    weights_test=weights_test,
    model=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=preprocess_dict,
    model_params=model_params,
    scorecard=scorecard,
    importance=importance,
)

 2025-05-12 23:43:14 | morai.forecast.metrics | INFO     | Adding model 'gbm' 


In [ ]:
charters.chart(
    df=importance, x_axis="feature", y_axis="importance", type="bar", y_sort=True
)

In [ ]:
charters.compare_rates(
    model_data,
    x_axis="attained_age",
    rates=["qx_raw", "qx_vbt15", "qx_gbm"],
    weights=["amount_exposed"],
    secondary="death_count",
)

 2025-05-12 23:43:15 | morai.experience.charters | INFO     | The weights list is 1 long and should be 3 long. Using the first weight for all weights. 


## XGBoost

**Overview:**
- An advanced implementation of gradient boosting designed for speed and performance.

**Feature Preprocessing:**
- XGBoost doesn't need all numeric variables and can handle categorical

In [ ]:
model_name = "xgb"

In [ ]:
preprocess_dict = preprocessors.preprocess_data(
    model_data,
    feature_dict=feature_dict,
    standardize=False,
    preset="pass",
)

 2025-05-12 23:43:15 | morai.forecast.preprocessors | INFO     | using 'pass' preset which makes all features passthrough 
 2025-05-12 23:43:15 | morai.forecast.preprocessors | INFO     | model target: ['qx_raw'] 
 2025-05-12 23:43:15 | morai.forecast.preprocessors | INFO     | model weights: ['amount_exposed'] 
 2025-05-12 23:43:16 | morai.forecast.preprocessors | INFO     | passthrough - (generally numeric): ['attained_age', 'observation_year', 'duration', 'sex', 'smoker_status', 'binned_face', 'class_enh', 'insurance_plan'] 


In [ ]:
X = preprocess_dict["X"]
y = preprocess_dict["y"]
weights = preprocess_dict["weights"]
mapping = preprocess_dict["mapping"]
md_encoded = preprocess_dict["md_encoded"]
model_features = preprocess_dict["model_features"]

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, random_state=0, test_size=0.2
)

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor

if model_build:
    model = XGBRegressor(
        objective="reg:squarederror",
        eval_metric="rmse",
        max_depth=6,
        eta=0.05,
        subsample=0.8,
        colsample_bytree=0.5,
        enable_categorical=True,
    )

    model.fit(X_train, y_train, sample_weight=weights_train)
if model_load:
    model = joblib.load(f"files/models/{model_name}.joblib")
    logger.info(f"loaded model '{model_name}'. type: {type(model)}")
if model_save:
    joblib.dump(model, f"files/models/{model_name}.joblib")
    logger.info(f"saved model '{model_name}'. type: {type(model)}")

model_params = {"weights": True}
model_params.update(model.get_params())

 2025-05-12 23:43:23 | __main__ | INFO     | saved model 'xgb'. type: <class 'xgboost.sklearn.XGBRegressor'> 


In [ ]:
predictions = model.predict(X)

In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
importance = base.ModelWrapper(model).get_importance()

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=y_train,
    y_pred_train=model.predict(X_train),
    weights_train=weights_train,
    y_true_test=y_test,
    y_pred_test=model.predict(X_test),
    weights_test=weights_test,
    model=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=preprocess_dict,
    model_params=model_params,
    scorecard=scorecard,
    importance=importance,
)

 2025-05-12 23:43:28 | morai.forecast.metrics | INFO     | Adding model 'xgb' 


In [ ]:
charters.chart(
    df=importance, x_axis="feature", y_axis="importance", type="bar", y_sort=True
)

In [ ]:
charters.compare_rates(
    model_data,
    x_axis="insurance_plan",
    rates=["qx_raw", "qx_xgb", "qx_vbt15"],
    weights=["amount_exposed"],
    secondary="death_count",
)

 2025-05-12 23:43:29 | morai.experience.charters | INFO     | The weights list is 1 long and should be 3 long. Using the first weight for all weights. 


In [ ]:
charters.target(
    df=model_data,
    target="ratio",
    cols=3,
    features=model_features,
    numerator=["death_claim_amount"],
    denominator=["exp_amt_xgb"],
).show()

 2025-05-12 23:43:29 | morai.experience.charters | INFO     | Creating '9' target plots. 


In [ ]:
# lgb.plot_tree(bst, tree_index=0, figsize=(20, 10))
# plt.show()

## CatBoost

**Overview**
- A CatBoost model is an ensemble of decision trees that are boosted. It is very good with non-linear data, but does not extrapolate well.

**Feature Preprocessing:**
- CatBoost doesn't need all numeric variables and can handle categorical

In [ ]:
model_name = "cat"

In [ ]:
preprocess_dict = preprocessors.preprocess_data(
    model_data,
    feature_dict=feature_dict,
    standardize=False,
    preset="pass",
)

 2025-05-12 23:43:30 | morai.forecast.preprocessors | INFO     | using 'pass' preset which makes all features passthrough 
 2025-05-12 23:43:30 | morai.forecast.preprocessors | INFO     | model target: ['qx_raw'] 
 2025-05-12 23:43:30 | morai.forecast.preprocessors | INFO     | model weights: ['amount_exposed'] 
 2025-05-12 23:43:30 | morai.forecast.preprocessors | INFO     | passthrough - (generally numeric): ['attained_age', 'observation_year', 'duration', 'sex', 'smoker_status', 'binned_face', 'class_enh', 'insurance_plan'] 


In [ ]:
X = preprocess_dict["X"]
y = preprocess_dict["y"]
weights = preprocess_dict["weights"]
mapping = preprocess_dict["mapping"]
md_encoded = preprocess_dict["md_encoded"]
model_features = preprocess_dict["model_features"]

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, random_state=0, test_size=0.2
)

In [ ]:
cat_features = feature_dict["ordinal"] + feature_dict["ohe"] + feature_dict["nominal"]
cat_features = list(set(cat_features) & set(model_features))

In [ ]:
from catboost import CatBoostRegressor

if model_build:
    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.05,
        depth=8,
        cat_features=cat_features,
        one_hot_max_size=11,
    )
    model.fit(X_train, y_train, sample_weight=weights_train, verbose=0)
if model_load:
    model = joblib.load(f"files/models/{model_name}.joblib")
    logger.info(f"loaded model '{model_name}'. type: {type(model)}")
if model_save:
    joblib.dump(model, f"files/models/{model_name}.joblib")
    logger.info(f"saved model '{model_name}'. type: {type(model)}")

model_params = {"weights": True}
model_params.update(model.get_params())

 2025-05-12 23:47:17 | __main__ | INFO     | saved model 'cat'. type: <class 'catboost.core.CatBoostRegressor'> 


In [ ]:
predictions = model.predict(X)

In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
importance = base.ModelWrapper(model).get_importance()

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=y_train,
    y_pred_train=model.predict(X_train),
    weights_train=weights_train,
    y_true_test=y_test,
    y_pred_test=model.predict(X_test),
    weights_test=weights_test,
    model=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=preprocess_dict,
    model_params=model_params,
    scorecard=scorecard,
    importance=importance,
)

 2025-05-12 23:47:20 | morai.forecast.metrics | INFO     | Adding model 'cat' 


In [ ]:
charters.chart(
    df=importance, x_axis="feature", y_axis="importance", type="bar", y_sort=True
)

In [ ]:
charters.compare_rates(
    model_data,
    x_axis="attained_age",
    rates=["qx_raw", "qx_cat", "qx_vbt15"],
    weights=["amount_exposed"],
    secondary="death_count",
)

 2025-05-12 23:47:20 | morai.experience.charters | INFO     | The weights list is 1 long and should be 3 long. Using the first weight for all weights. 


In [ ]:
charters.target(
    df=model_data,
    target="ratio",
    cols=3,
    features=model_features,
    numerator=["death_claim_amount"],
    denominator=["exp_amt_cat"],
).show()

 2025-05-12 23:47:22 | morai.experience.charters | INFO     | Creating '9' target plots. 


## Lee Carter

The Lee-Carter model is formulated as follows:

$$\log(m_{x,t}) = a_{x} + b_{x}k_{t} + \epsilon_{x,t}$$

source: https://en.wikipedia.org/wiki/Lee%E2%80%93Carter_model

Where:
- $m_{x,t}$ is the _matrix mortality rate_ at age $x$ in year $t$
- $a_{x}$ describes the general _shape_ of mortality at age $x$ (mean of the time-averaged logs of the central mortality rate at age $x$). This is known as the _age effect_ on mortality.
- $b_{x}$ measures the change in the rates at age $x$ due to a change in the underlying time trend, $k_{t}$.
- $k_{t}$ reflects the effect of the time trend, $t$, on mortality.
- $\epsilon_{x,t}$ are i.i.d. normal random variables with zero means and constant variance, $\sigma^{2}$.

In [ ]:
model_name = "lc"

In [ ]:
model_params = {"weights": True}
model = base.LeeCarter()

 2025-05-13 00:02:31 | morai.models.base | INFO     | initialized LeeCarter 


In [ ]:
lc_df = model.structure_df(model_data)

 2025-05-13 00:02:31 | morai.models.base | INFO     | grouping data by age and year 
 2025-05-13 00:02:31 | morai.models.base | INFO     | calculating qx_raw rates using death_claim_amount and amount_exposed 
 2025-05-13 00:02:31 | morai.models.base | INFO     | floored 50 rates to 0.000001 and capped 0 rates to 0.999999. 
 2025-05-13 00:02:31 | morai.models.base | INFO     | crude_df shape: (488, 5) 


In [ ]:
lc_df = model.fit(lc_df)

 2025-05-13 00:02:31 | morai.models.base | INFO     | creating Lee Carter model with qx_raw rates... 
 2025-05-13 00:02:31 | morai.models.base | INFO     | age range: 50, 113 
 2025-05-13 00:02:31 | morai.models.base | INFO     | year range: 2012, 2019 
 2025-05-13 00:02:31 | morai.models.base | INFO     | creating `1` intervals 
 2025-05-13 00:02:31 | morai.models.base | INFO     | adding qx_lc to lc_df 


In [ ]:
model_data = model.map(model_data)

 2025-05-13 00:02:32 | morai.models.base | INFO     | mapping qx_lc to df 


In [ ]:
lcf_df = model.forecast(years=5)

 2025-05-13 00:02:33 | morai.models.base | INFO     | forecasting qx_lc using deterministic random walk... 


In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=model_data["death_claim_amount"],
    y_pred_train=model_data[f"exp_amt_{model_name}"],
    weights_train=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=None,
    model_params=model_params,
    scorecard=scorecard,
    importance=None,
)

 2025-05-13 00:02:33 | morai.forecast.metrics | ERROR    | Model name lc already exists, not adding model. Please use a different name. 


Average log qx shows that mortality rates increase by age

In [ ]:
charters.chart(
    df=pd.DataFrame(model.a_x).set_axis([0], axis=1).reset_index(),
    x_axis="attained_age",
    y_axis=0,
    color=None,
    type="line",
    title="Average of log(qx)",
    labels={"attained_age": "attained_age", "0": "log_qx"},
)

In [ ]:
charters.chart(
    df=pd.DataFrame(model.k_t).set_axis([0], axis=1).reset_index(),
    x_axis="observation_year",
    y_axis=0,
    color=None,
    type="line",
    title="Time Trend of Mortality",
    labels={"observation_year": "observation_year", "0": "k_t"},
)

In [ ]:
charters.chart(
    df=pd.DataFrame(model.b_x).set_axis([0], axis=1).reset_index(),
    x_axis="attained_age",
    y_axis=0,
    color=None,
    type="line",
    title="Change in the Rate per Age with Respect to each Year",
    labels={"attained_age": "attained_age", "0": "b_x"},
)

In [ ]:
charters.compare_rates(
    model_data,
    x_axis="observation_year",
    rates=["qx_raw", "qx_lc"],
    weights=["amount_exposed"],
    secondary="amount_exposed",
)

 2025-05-12 23:50:23 | morai.experience.charters | INFO     | The weights list is 1 long and should be 2 long. Using the first weight for all weights. 


## Cairns-Blake-Dowd Model

The Cairns-Blake-Dowd (CBD) model is a stochastic mortality model, and a variant of the Lee-Carter mortality model. It is designed with longevity risk in mind, particularly attempting to understand and forecast mortality at older ages.

The model is in the following form:

$$\text{logit}(_{t}q_{x}) = \kappa_{t}^{(1)} + (x - \bar{x})\kappa_{t}^{(2)}$$

- $\text{logit}(\alpha) = \log\big(\frac{\alpha}{1-\alpha}\big)$
- $\kappa_{t}^{(1)}$  is the _level_ factor, varying with respect to year $t \in (t_{1}, t_{2}, \dots, t_{q})$
- $\kappa_{t}^{(2)}$  is the _slope_ factor, varying with respect to year $t \in (t_{1}, t_{2}, \dots, t_{q})$
- $x$ is the age group, $x \in (x_{1}, x_{2}, \dots, x_{p})$
- $\bar{x}$ is the average of the age group
- $_{t}q_{x}$ is the probability an individual aged $x$ last birthday dies before age $x+t$

We will be using method of least-squares to get estimates $\hat{\kappa}^{(1)}$ and $\hat{\kappa}^{(2)}$.

Further information:
- https://www.actuaries.org/AFIR/Colloquia/Rome2/Cairns_Blake_Dowd.pdf - further discussion on the merits of the CBD model (such as age-period cohort effects and further extensions to the CBD model).

In [ ]:
model_name = "cbd"

In [ ]:
model_params = {"weights": True}
model = base.CBD()

 2025-05-12 23:54:11 | morai.models.base | INFO     | initialized CBD 


In [ ]:
cbd_df = model.structure_df(model_data)

 2025-05-12 23:54:11 | morai.models.base | INFO     | grouping data by age and year 
 2025-05-12 23:54:11 | morai.models.base | INFO     | calculating qx_raw rates using death_claim_amount and amount_exposed 
 2025-05-12 23:54:11 | morai.models.base | INFO     | floored 50 rates to 0.000001 and capped 0 rates to 0.999999. 
 2025-05-12 23:54:11 | morai.models.base | INFO     | cbd_df shape: (488, 5) 


In [ ]:
cbd_df = model.fit(cbd_df)

 2025-05-12 23:54:11 | morai.models.base | INFO     | creating CBD model with qx_raw rates... 
 2025-05-12 23:54:11 | morai.models.base | INFO     | age range: 50, 113 
 2025-05-12 23:54:11 | morai.models.base | INFO     | year range: 2012, 2019 
 2025-05-12 23:54:11 | morai.models.base | INFO     | creating `1` intervals 
 2025-05-12 23:54:11 | morai.models.base | INFO     | adding qx_cbd to cbd_df 


In [ ]:
model_data = model.map(model_data)

 2025-05-12 23:54:12 | morai.models.base | INFO     | mapping qx_cbd to df 


In [ ]:
cbdf_df = model.forecast(years=5)

 2025-05-12 23:54:13 | morai.models.base | INFO     | forecasting qx_cbd using deterministic random walk... 


In [ ]:
model_data = experience.calc_qx_exp_ae(
    model_data=model_data,
    predictions=predictions,
    model_name=model_name,
    exposure_col="amount_exposed",
    actual_col="death_claim_amount",
)

In [ ]:
scorecard = model_results.get_scorecard(
    y_true_train=model_data["death_claim_amount"],
    y_pred_train=model_data[f"exp_amt_{model_name}"],
    weights_train=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=None,
    model_params=model_params,
    scorecard=scorecard,
    importance=None,
)

 2025-05-12 23:54:15 | morai.forecast.metrics | INFO     | Adding model 'cbd' 


In [ ]:
# shape shows the impact of time on mortality
charters.chart(
    df=pd.concat([pd.DataFrame(model.k_t_1).set_axis([0], axis=1), pd.DataFrame(model.k_1_f).set_axis([0], axis=1)]).reset_index(),
    x_axis=model.year_col,
    y_axis=0,
    color=None,
    type="line",
    title="Shape Component by year",
    labels={"0": "k_t_1"},
)

In [ ]:
# slope shows if the impact of time is different by year
charters.chart(
    df=pd.concat([pd.DataFrame(model.k_t_2).set_axis([0], axis=1), pd.DataFrame(model.k_2_f).set_axis([0], axis=1)]).reset_index(),
    x_axis=model.year_col,
    y_axis=0,
    color=None,
    type="line",
    title="Slope Component by year",
    labels={"0": "k_t_2"},
)

In [ ]:
charters.compare_rates(
    model_data,
    x_axis="observation_year",
    rates=["qx_raw", "qx_cbd"],
    weights=["amount_exposed"],
    secondary="amount_exposed",
)

# Model Results

In [ ]:
original_float_format = pd.options.display.float_format
pd.options.display.float_format = "{:,.6f}".format  # Increase to 10 decimal places
model_results.scorecard.sort_values(by=("test", "r2_score"), ascending=False)

model_name    train                                                      \
                    ae    smape  r2_score root_mean_squared_error    shape   
5         gbm 1.000000 1.926381  0.130311          208,125.455513   877381   
4         gam 1.000000 1.924616  0.101464          211,548.931163   877381   
10        glm 1.000000 1.924590  0.096490          212,133.664346   877381   
6         xgb 1.000564 1.924258  0.189317          200,941.049357   877381   
2        tree 1.006885 1.928945  0.090978          212,779.831784   877381   
3          rf 1.007973 1.917206  0.073377          214,829.864796   877381   
7         cat 1.000031 1.922730  0.441500          166,784.091283   877381   
1          lr 1.004714 1.930634 -0.029726          226,466.484193   877381   
0       vbt15 0.898235 1.926877  0.097066          206,603.606989  1096727   
8          lc 0.998724 1.923082  0.381053          171,055.283069  1096727   
9         cbd 0.998724 1.923082  0.381053          171,055.283069  1096727   

       test                                                            \
         ae    smape  r2_score root_mean_squared_error          shape   
5  0.990595 1.927327  0.130525          179,706.464012 219,346.000000   
4  0.985262 1.925667  0.124703          180,307.107580 219,346.000000   
10 0.983754 1.925512  0.121722          180,613.821923 219,346.000000   
6  0.989847 1.925081  0.118889          180,904.935748 219,346.000000   
2  0.991037 1.929929  0.085698          184,280.751559 219,346.000000   
3  0.991936 1.918309  0.071365          185,719.511591 219,346.000000   
7  0.993435 1.924492  0.056825          187,167.831566 219,346.000000   
1  0.981576 1.930753 -0.041418          196,674.257615 219,346.000000   
0       NaN      NaN       NaN                     NaN            NaN   
8       NaN      NaN       NaN                     NaN            NaN   
9       NaN      NaN       NaN                     NaN            NaN   

                    train                   test  
                      aic                    aic  
5                     NaN                    NaN  
4                     NaN                    NaN  
10 150,782,682,356.605621 150,782,682,356.605621  
6                     NaN                    NaN  
2                     NaN                    NaN  
3                     NaN                    NaN  
7                     NaN                    NaN  
1                     NaN                    NaN  
0                     NaN                    NaN  
8                     NaN                    NaN  
9                     NaN                    NaN

In [ ]:
pd.options.display.float_format = original_float_format

In [ ]:
break

In [ ]:
# write out results
model_results.save_model()
model_data.to_parquet("files/dataset/model_data.parquet")

 2025-05-13 00:07:17 | morai.forecast.metrics | INFO     | saving results to C:\Users\johnk\Desktop\github\morai\files\result\model_results.json 


In [ ]:
# model_results = metrics.ModelResults(filepath='model_results.json')

In [ ]:
rank_df = metrics.ae_rank(
    df=model_data[model_data["insurance_plan"].isin(["UL", "ULSG"])],
    features=model_features,
    actuals="death_claim_amount",
    expecteds="exp_amt_vbt15",
    exposures="amount_exposed",
)
rank_df.head()

# Visualizations

## Chart

In [ ]:
charters.chart(
    df=model_data,
    x_axis="attained_age",
    y_axis="ratio",
    color=None,
    type="line",
    numerator="death_claim_amount",
    denominator="amount_exposed",
)

 2025-05-13 00:07:29 | morai.experience.charters | INFO     | Calculating ratio using [death_claim_amount] and [amount_exposed] 


In [ ]:
charters.chart(
    df=model_data,
    x_axis="duration",
    y_axis="ratio",
    color="face_amount_band",
    numerator="death_claim_amount",
    denominator="amount_exposed",
    x_bins=5,
    type="line",
)

 2025-05-13 00:07:33 | morai.experience.charters | INFO     | Calculating ratio using [death_claim_amount] and [amount_exposed] 
 2025-05-13 00:07:33 | morai.experience.charters | INFO     | Binning feature: [duration] with 5 bins 


In [ ]:
charters.chart(
    df=model_data[
        (model_data["attained_age"] > 50)
        & (model_data["attained_age"] < 90)
        & (model_data["insurance_plan"].isin(["Term", "UL", "Perm", "ULSG"]))
    ],
    x_axis="duration",
    color="insurance_plan",
    y_axis="ratio",
    type="line",
    numerator="death_claim_amount",
    denominator="amount_exposed",
    x_bins=5,
)

 2025-05-13 00:07:36 | morai.experience.charters | INFO     | Calculating ratio using [death_claim_amount] and [amount_exposed] 
 2025-05-13 00:07:36 | morai.experience.charters | INFO     | Binning feature: [duration] with 5 bins 


This shows that as duration goes up the class structure becomes more NA preferred class structure which has poor a/e

## Compare

In [ ]:
charters.compare_rates(
    df=model_data[model_data["insurance_plan"].isin(["UL", "ULSG"])],
    x_axis="class_enh",
    rates=["ae_vbt15"],
    weights=["exp_amt_vbt15"],
    secondary="death_count",
    display=True,
)

In [ ]:
charters.chart(
    df=model_data[model_data["insurance_plan"].isin(["UL", "ULSG"])],
    x_axis="duration",
    y_axis="death_count",
    color="number_of_pfd_classes",
    x_bins=5,
    type="bar",
)

This shows the product sales by year

In [ ]:
charters.chart(
    df=model_data,
    x_axis="issue_year",
    y_axis="policies_exposed",
    color="insurance_plan",
    type="bar",
)

In [ ]:
charters.chart(
    df=model_data,
    x_axis="observation_year",
    y_axis="policies_exposed",
    color="number_of_pfd_classes",
    type="bar",
)

In [ ]:
# charters.scatter(df=df, target="qx_raw", numeric=True, sample_nbr=1000).show()

## PDP

In [ ]:
charters.pdp(
    model=GLM.model,
    df=md_encoded,
    x_axis="insurance_plan",
    mapping=mapping,
    weight="amount_exposed",
)

This shows that the perm product is not too high when accounting for the other variables

In [ ]:
charters.pdp(
    model=model,
    df=md_encoded,
    x_axis="insurance_plan",
    weight="amount_exposed",
    secondary="death_count",
    mapping=mapping,
    display=True,
)

This shows that for a 4 class system there seems to be wear-off of underwriting, but does persist in later ages

In [ ]:
charters.pdp(
    model=model,
    df=md_encoded[
        (
            md_encoded["class_enh"].isin(["2_1", "2_2"])
            & (md_encoded["insurance_plan"] == "UL")
            & (md_encoded["attained_age"] >= 65)
            & (md_encoded["attained_age"] <= 90)
        )
    ],
    x_axis="attained_age",
    line_color="class_enh",
    weight="amount_exposed",
    mapping=mapping,
    center="per_x",
    secondary="death_count",
    display=True,
    n_jobs=-1,
)

In [ ]:
charters.pdp(
    model=model,
    df=md_encoded[
        (
            md_encoded["class_enh"].isin(["2_1", "2_2"])
            & (md_encoded["insurance_plan"] == "UL")
            & (md_encoded["duration"] <= 30)
        )
    ],
    x_axis="duration",
    line_color="class_enh",
    weight="amount_exposed",
    secondary="death_count",
    mapping=mapping,
    center="per_x",
    display=True,
    n_jobs=-1,
)

This shows there may be something with the data that there is an uptick in 2019. It also shows a similar pattern for all the products. Granted it is a random forest model so there may be some influence in the prediction

In [ ]:
charters.pdp(
    model=model,
    df=md_encoded,
    x_axis="observation_year",
    line_color="insurance_plan",
    weight="amount_exposed",
    secondary="death_count",
    mapping=mapping,
    display=True,
    n_jobs=-1,
)

# Reload

In [ ]:
importlib.reload(charters)

In [ ]:
importlib.reload(custom_logger)

In [ ]:
importlib.reload(models)

<module 'morai.forecast.models' from 'C:\\Users\\johnk\\Desktop\\github\\morai\\morai\\forecast\\models.py'>

In [ ]:
importlib.reload(helpers)

<module 'morai.utils.helpers' from 'C:\\Users\\johnk\\Desktop\\github\\morai\\morai\\utils\\helpers.py'>

In [ ]:
importlib.reload(metrics)

<module 'morai.forecast.metrics' from 'C:\\Users\\johnk\\Desktop\\github\\morai\\morai\\forecast\\metrics.py'>

In [ ]:
importlib.reload(preprocessors)

<module 'morai.forecast.preprocessors' from 'C:\\Users\\johnk\\Desktop\\github\\morai\\morai\\forecast\\preprocessors.py'>

# Utilities

In [ ]:
jupyter_objects = helpers.memory_usage_jupyter()
jupyter_objects

,object,size_mb
0,model_data,394.32
1,md_encoded,388.04
2,X,49.16
3,X_train,46.02
4,weights_train,13.39
...,...,...
60,model_name,0.00
61,initial_row_count,0.00
62,model_save,0.00
63,model_load,0.00


In [ ]:
helpers.delete_jupyter_objects(objects=list(jupyter_objects["object"]))

# Unused Models

## LightGBM

**Feature Preprocessing:**
- LightGBM doesn't need all numeric variables and can handle categorical

In [ ]:
model_name = "lgb"

In [ ]:
preprocess_dict = preprocessors.preprocess_data(
    model_data,
    feature_dict=feature_dict,
    standardize=False,
    preset="pass",
)

In [ ]:
X = preprocess_dict["X"]
y = preprocess_dict["y"]
weights = preprocess_dict["weights"]
mapping = preprocess_dict["mapping"]
md_encoded = preprocess_dict["md_encoded"]
model_features = preprocess_dict["model_features"]

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    X, y, weights, random_state=0, test_size=0.2
)

In [ ]:
import lightgbm as lgb

if model_build:
    params = {
        "boosting_type": "gbdt",
        "objective": "regression",
        "metric": "rmse",
        "num_leaves": 31,
        "learning_rate": 0.05,
        "feature_fraction": 0.5,
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
    }

    train_data = lgb.Dataset(X_train, label=y_train, weight=weights_train)
    model = lgb.train(params, train_data)
if model_load:
    model = joblib.load(f"files/models/{model_name}.joblib")
    logger.info(f"loaded model '{model_name}'. type: {type(model)}")
if model_save:
    joblib.dump(model, f"files/models/{model_name}.joblib")
    logger.info(f"saved model '{model_name}'. type: {type(model)}")

model_params = {"weights": True}
model_params.update(model.params)

In [ ]:
predictions = model.predict(X)

In [ ]:
model_data = calc_qx_exp_ae(model_data, predictions, model_name)

In [ ]:
importance = get_importance(
    features=X.columns, values=model.feature_importance(importance_type="gain")
)

In [ ]:
# get results
scorecard = model_results.get_scorecard(
    y_true_train=y_train,
    y_pred_train=model.predict(X_train),
    weights_train=weights_train,
    y_true_test=y_test,
    y_pred_test=model.predict(X_test),
    weights_test=weights_test,
    model=None,
)
model_results.add_model(
    model_name=model_name,
    data_path=pl_parquet_path,
    data_shape=model_data.shape,
    preprocess_dict=preprocess_dict,
    model_params=model_params,
    scorecard=scorecard,
    importance=importance,
)

In [ ]:
charters.chart(
    df=importance, x_axis="feature", y_axis="importance", type="bar", y_sort=True
)

In [ ]:
charters.compare_rates(
    model_data[model_data["insurance_plan"].isin(["UL"])],
    x_axis="insurance_plan",
    rates=["qx_raw", "qx_lgb", "qx_vbt15"],
    weights=["amount_exposed"],
    secondary="death_count",
)

In [ ]:
charters.target(
    df=model_data,
    target="ratio",
    cols=3,
    features=model_features,
    numerator=["death_claim_amount"],
    denominator=["exp_amt_lgb"],
).show()

In [ ]:
# lgb.plot_tree(model, tree_index=0, figsize=(20, 10))
# plt.show()

# Test

In [ ]:
charters.compare_rates(
    df=model_data,
    x_axis="attained_age",
    rates=["qx_vbt15", "qx_glm"],
    weights=["amount_exposed"],
    secondary="death_count",
    display=True,
)

In [ ]:
import plotly.io as pio

print(pio.renderers.default)

plotly_mimetype+notebook


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

# pio.renderers.default = "jpeg"
# notebook_connected
# jupyterlab

fig = go.Figure(data=go.Bar(y=[2, 3, 1]))
fig.show()